# Example: Prospect Theory

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('bmh')
import matplotlib
matplotlib.rcParams.update({'font.size': 22})
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_probability as tfp
import scipy
import seaborn
from tqdm import tqdm
import yfinance as yf
from scipy.stats import norm
from optimization_functions import *
%load_ext autoreload
%autoreload 2
%aimport optimization_functions
    

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Download Stock Prices and create Training Periods

In [16]:
stocks = ["GOOGL","EBAY","AMZN","MSFT","AAPL"]
Returns_train_tf_dict = {}

In [17]:
for s in stocks:
    S_train = yf.download(s, start="2010-01-02", end="2020-02-01")
    S_train = S_train.dropna()
    S_train = S_train.reset_index()
    S_train = S_train[["Date","Close"]]
    
    Returns_train = S_train["Close"].diff() / S_train["Close"].shift(1)
    Returns_train = Returns_train.dropna()
    # A Tensorflow Adaption
    Returns_train_tf_dict[s] = tf.reshape(Returns_train,(1,len(Returns_train)))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Test Period 1: Feb 2020 - May 2020

# Description of the Algorithm

For $t = T-1, \cdots , 1$ :


For $j = 1,\dots, N_{iterations}$:

Sample $(\omega_1,\dots,\omega_t)=\omega^t$

Sample $(a_0,\dots,a_{t-1})=a^t$

For $k = 1,\dots, N_{measures}$:

Sample next states
$w^{t+1, (k),(i)} \sim \mathbb{P}_k \in \mathcal{P}_t(\omega^t)$ for $i= 1,\dots,N_{MC}$

<!-- Compute $\tilde{J}_t(\omega,a^t, \tilde{a}) = \min_k \frac{1}{N}\sum_{i=1} \tilde{\Psi}_{t+1}((\omega^t,a^t, \tilde{a}))$
Maximize w.r.t. $\widetilde{a}$:

$\widetilde{a}_t(\omega^t,a^t)=argmax_{\widetilde{a}} \tilde{J}_t(\omega,a^t, \tilde{a})$

Approximate

$\tilde{\Psi}_t(\omega^t, a^t):= \tilde{J}_t(\omega,a^t, \widetilde{a}_t(\omega^t,a^t))$ -->

The optimization step is:

1.)

Maximize
$$
\sup_{\widetilde{a} \in A}\inf_{ P \in \mathcal{P}_t(\omega^t)} E_P \left[\Psi_{t+1}\left(\omega^t \otimes_t \cdot, (a^{t},\widetilde{a})\right)\right]= \Psi_t(\omega^t,a^t)
$$
 w.r.t. parameters of $\tilde{a}_t$:

 Alternative:

 By using a dual approach we have in the special case of a Wasserstein distance
 $$
\Psi_t(\omega^t,a^t)=\sup_{\widetilde{a} \in A}\inf_{ P \in \mathcal{P}_t(\omega^t)} E_P \left[\Psi_{t+1}\left(\omega^t \otimes_t \cdot, (a^{t},\widetilde{a})\right)\right]
$$
$$
= \sup_{\widetilde{a} \in A}\sup_{ \lambda \geq 0} E_{\widehat{P}} \left[\inf_{z} \left\{\Psi_{t+1}\left(\omega^t \otimes_t z, (a^{t},\widetilde{a})\right)+\lambda \|\cdot -z\|\right\}\right] - \lambda \varepsilon
$$
$$
= \sup_{\widetilde{a} \in A}\sup_{ \lambda \geq 0} \frac{1}{N_{MC}} \sum_{i=1}^{N_{MC}} \left[\min_{j=1,\dots,N} \left\{\Psi_{t+1}\left(\omega^t \otimes_t z_j, (a^{t},\widetilde{a})\right)+\lambda \|x_i -z_j\|\right\}\right] - \lambda \varepsilon
$$
$$
= \sup_{\widetilde{a} \in A}\sup_{ \lambda \geq 0}  \sum_{i=t}^{N-1} \left[\pi_i(\omega^t) \min_{j=1,\dots,N} \left\{\Psi_{t+1}\left(\omega^t \otimes_t z_j, (a^{t},\widetilde{a})\right)+\lambda \|\mathcal{R}_{i+1} -z_j\|\right\}\right] - \lambda \varepsilon
$$
2.)


Minimize
$$
\sum_{b=1}^B \left(\tilde{\Psi}_t(\omega^t_b, a^t_b)- \left(\sup_{\widetilde{a} \in A}\sup_{ \lambda \geq 0}  \sum_{i=t}^{N-1} \left[\pi_i(\omega^t) \min_{j=1,\dots,N} \left\{\Psi_{t+1}\left(\omega^t \otimes_t z_j, (a^{t},\widetilde{a})\right)+\lambda \|\mathcal{R}_{i+1} -z_j\|\right\}\right] - \lambda \varepsilon \right)  \right)^2
$$
 w.r.t. parameters of $\tilde{\Psi}_t$:

Finally, we set:
$$
\Omega^t \ni \omega^t =(\omega_1,\dots,\omega_{t}) \mapsto a_t^*(\omega^t):= \widetilde{a}_t^*\left(\omega^t,~\left(a_0^*,\dots,a_{t-1}^*(\omega_1,\dots,\omega_{t-1}\right)\right) \in A.
$$

# Training

Number of periods

In [18]:
T = 10

Initialize the dictionaries

In [19]:
a_tilde_dict = {}
a_tilde_robust_0001_dict = {}
a_tilde_robust_0005_dict = {}

Set the parameters

In [20]:
# Number of iterations per epoch
inner_psi = 2000
inner_a = 500
# Tolerance levels to break the loop
tolerance_psi = 0.00001
tolerance_a = 0.000001
# Leraning rates
learning_rate_Psi = 0.001
learning_rate_a = 0.001
learning_rate_LAM = 0.001
# Number of Monte Carlo Simulations
N_MC = 128
N_MC_inner = 128
#Batch Sizes
Batch_size_a = 128
Batch_size_psi = 128

Train

In [21]:
for s in stocks:


    #compute
    print("Start optimization for Stock: {}".format(s))
    print("Robust, epsilon = 0.0005")
    a_tilde_robust_0005_dict[s], psi = train_networks(Returns_train_tf_dict[s],T=T,inner_psi = inner_psi,
                       inner_a =inner_a,  N_MC=N_MC, N_MC_inner=N_MC_inner, epsilon = 0.0005,tolerance_psi =tolerance_psi,
                       tolerance_a = tolerance_a, learning_rate_Psi = learning_rate_Psi,learning_rate_a = learning_rate_a,
                                        learning_rate_LAM = learning_rate_LAM,
                                                    Batch_size_a = Batch_size_a,
                                        Batch_size_psi = Batch_size_psi,print_intermediate_results = True)   
    #save
    file_save(a_tilde_robust_0005_dict[s],s,"Robust_0005")
    file_save(psi[:T],s,"Psi_Robust_0005")
    
    #compute
    print("Start optimization for Stock: {}".format(s))
    print("Robust, epsilon = 0.0001")
    a_tilde_robust_0001_dict[s], psi = train_networks(Returns_train_tf_dict[s],T=T,inner_psi = inner_psi,
                       inner_a =inner_a, N_MC=N_MC, N_MC_inner=N_MC_inner,
                       epsilon = 0.0001,tolerance_psi =tolerance_psi,
                       tolerance_a = tolerance_a, learning_rate_Psi = learning_rate_Psi,learning_rate_a = learning_rate_a,
                                        learning_rate_LAM = learning_rate_LAM,Batch_size_a = Batch_size_a,
                                        Batch_size_psi = Batch_size_psi,print_intermediate_results = True)   
    #save
    file_save(a_tilde_robust_0001_dict[s],s,"Robust_0001")
    file_save(psi[:T],s,"Psi_Robust_0001")    
    #compute
    print("Start optimization for Stock: {}".format(s))
    print("Non-Robust")
    a_tilde_dict[s], psi = train_networks(Returns_train_tf_dict[s],T=T,inner_psi = inner_psi,
                       inner_a =inner_a, N_MC=N_MC, N_MC_inner=N_MC_inner,tolerance_psi =tolerance_psi,
                       tolerance_a = tolerance_a, learning_rate_Psi = learning_rate_Psi,learning_rate_a = learning_rate_a,
                                        learning_rate_LAM = learning_rate_LAM,
                                        Batch_size_a = Batch_size_a,
                                        Batch_size_psi = Batch_size_psi,
                                        print_intermediate_results = True) 
    #save
    file_save(a_tilde_dict[s],s,"Non-Robust")
    file_save(psi[:T],s,"Psi_Non-Robust")


Start optimization for Stock: GOOGL
Robust, epsilon = 0.0005
Start Backwards Iterations
#########
# t = 9 #
#########

a: -0.926754355430603, Iteration: 0
a: -0.8903558850288391, Iteration: 25
a: -0.8910620808601379, Iteration: 50
a: -0.8719322085380554, Iteration: 75
a: -0.9015124440193176, Iteration: 100
a: -0.888285756111145, Iteration: 125
a: -0.8829493522644043, Iteration: 150
a: -0.8834388256072998, Iteration: 175
a: -0.8646166324615479, Iteration: 200
a: -0.8854440450668335, Iteration: 225
a: -0.8742781281471252, Iteration: 250
a: -0.8773781061172485, Iteration: 275
a: -0.8528931140899658, Iteration: 300
a: -0.8707038164138794, Iteration: 325
a: -0.8634989857673645, Iteration: 350
a: -0.8742594122886658, Iteration: 375
a: -0.8597755432128906, Iteration: 400
a: -0.8656796216964722, Iteration: 425
a: -0.8742989301681519, Iteration: 450
a: -0.8819112181663513, Iteration: 475
Psi: 0.9188849329948425, Iteration: 0
Psi: 0.538306474685669, Iteration: 25
Psi: 0.18781647086143494, Iterat